In [5]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as py
py.init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings("ignore")
from scipy.stats import ttest_ind, ttest_1samp
import gc

SMOOTHING = 3

In [14]:
def clean_data(df):
    if "Unnamed: 0" in df.columns:
        df.drop("Unnamed: 0", axis=1, inplace=True)
        
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.rstrip('%').str.replace(',', '', regex=True)
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                pass
        
    df.set_index(df.columns[0], inplace=True)
    
    return df

for tier in range(3, 22):  # Loop through tiers 3 to 21
    file_path = f"map_data/maps_competitive_tier={tier}.csv"  # Generate file path for each tier

    try:
        # Read the CSV file for each tier
        df = pd.read_csv(file_path)
        # Clean the data using the clean_data function
        cleaned_df = clean_data(df)
        # Perform operations or analysis on the cleaned DataFrame if needed
        # For example, you can print the first few rows of each cleaned DataFrame
        print(f"Tier {tier}:")
        print(cleaned_df.head())

    except FileNotFoundError:
        print(f"File not found for tier {tier}")


Tier 3:
Empty DataFrame
Columns: [Play Rate, Atk Win, Def Win, Num Matches]
Index: []
Tier 4:
Empty DataFrame
Columns: [Play Rate, Atk Win, Def Win, Num Matches]
Index: []
Tier 5:
Empty DataFrame
Columns: [Play Rate, Atk Win, Def Win, Num Matches]
Index: []
Tier 6:
Empty DataFrame
Columns: [Play Rate, Atk Win, Def Win, Num Matches]
Index: []
Tier 7:
Empty DataFrame
Columns: [Play Rate, Atk Win, Def Win, Num Matches]
Index: []
Tier 8:
Empty DataFrame
Columns: [Play Rate, Atk Win, Def Win, Num Matches]
Index: []
Tier 9:
Empty DataFrame
Columns: [Play Rate, Atk Win, Def Win, Num Matches]
Index: []
Tier 10:
          Play Rate  Atk Win  Def Win  Num Matches
Map Name                                          
Breeze         13.6     51.9     48.1      17391.0
Haven          16.7     51.2     48.8      21267.0
Icebox         16.6     51.1     48.9      21216.0
Ascent         18.6     49.0     51.0      23715.0
Split          16.9     47.9     52.1      21573.0
Tier 11:
Empty DataFrame
Columns

In [11]:

general_ranks_list = ["Iron", "Bronze", "Silver", "Gold", "Platinum", "Diamond"]
ranks_list = np.array([f"{rank} {num}" for rank in general_ranks_list for num in range(1, 4)])
num_ranks = len(ranks_list)
print(f"Ranks: {ranks_list}, length: {num_ranks}")

Ranks: ['Iron 1' 'Iron 2' 'Iron 3' 'Bronze 1' 'Bronze 2' 'Bronze 3' 'Silver 1'
 'Silver 2' 'Silver 3' 'Gold 1' 'Gold 2' 'Gold 3' 'Platinum 1'
 'Platinum 2' 'Platinum 3' 'Diamond 1' 'Diamond 2' 'Diamond 3'], length: 18
